In [1]:
import requests
import bs4
import re
import pandas as pd

In [2]:
def get_html(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    res.raise_for_status()
    return res.text

In [3]:
ARTIST = "The Gaslight Anthem"
BASE_URL = "https://songmeanings.com/"
ARTIST_URL = BASE_URL + "artist/view/discography/137438979128/"
html = get_html(ARTIST_URL)

In [4]:
soup = bs4.BeautifulSoup(html, "lxml")

In [17]:
t1 = type(soup.select("div.album-holder.albums-list"))
t2 = type(soup.find_all("div", class_="album-holder albums-list")[0])

In [18]:
print(f"{t1}, {t2}")

<class 'list'>, <class 'bs4.element.Tag'>


In [37]:
e = soup.find_all("div", class_="album-holder albums-list")[0]

In [38]:
album_title = e.find("h3").getText()
album_title

'American Slang'

In [42]:
link = e.find("a").get_attribute_list("href")[0]
link

'/albums/view/tracks/247039/'

In [49]:
date = e.find("ul", class_="albumlist").find_all("li")[0].getText()
date

'2010-06-15'

In [51]:
date[:4]

'2010'

In [62]:
def get_albums(soup):
    albums = soup.find_all("div", class_="album-holder albums-list")
    df = pd.DataFrame()
    for album in albums:
        title = album.find("h3").getText()
        year = album.find("ul", class_="albumlist").find("li").getText()[:4]
        link = album.find("a").get_attribute_list("href")[0]
        series = pd.Series({"Title": title, "Year": year, "Link": link})
        df = df.append(series, ignore_index=True)
    return df.sort_values(by="Year")

In [63]:
df = get_albums(soup)

In [64]:
df["Link"] = BASE_URL[:-1] + df["Link"]

In [83]:
def get_songs(albums):
    df = pd.DataFrame()
    for idx, row in albums.iterrows():
        link = row["Link"]
        html = get_html(link)
        soup = bs4.BeautifulSoup(html, "lxml")
        songs = soup.find("table").find_all("a")
        for song in songs:
            link = song.get_attribute_list("href")[0]
            title = song.get_attribute_list("title")[0]
            series = pd.Series({"Title": title, 
                                "Link": link,
                                "Album": row["Title"],
                                "Year": row["Year"]})
            df = df.append(series, ignore_index=True)
    return df        

In [85]:
songs = get_songs(df)

In [97]:
songs["Link"] = "https:" + songs["Link"]

In [106]:
def get_lyrics(link_to_song):
    html = get_html(link_to_song)
    soup = bs4.BeautifulSoup(html, "lxml")
    lyrics = soup.find("div", class_="holder lyric-box").getText().strip()
    end = lyrics.find("Edit Lyrics")
    return text[:end]    

In [ ]:
songs["Lyrics"] = songs["Link"].apply(get_lyrics)

In [96]:
song_link = songs["Link"].iloc[10]

In [99]:
html = get_html(song_link)
soup = bs4.BeautifulSoup(html, "lxml")

"\n\t\t\t\t\t\tLet it ride, let it ride, let it roll off your mind\nDon't say a single world, get back inside\nLet me drive, let me drive, disappear in the night\nLike I was a ghost in your dreams\n\nAnd it was all completely wrong, Virginia\nYou were much too much for me\nI was a boy on the lawn, with his eyes to the sky\nYou were Garbo, smoke, and deceit\n\nAnd it was Saturday, September, and fire\nIt was a backdrop set in your smile\nIt was all I could want in the sway of the song\nI was haunted for so many years\n\nLet it ride, let it ride, let it roll off your mind\nDon't say a single world, get back inside\nLet me drive, let me drive, disappear in the night\nLike I was a ghost in your dreams\n\nSay goodbye, say goodbye, give a kiss and a sigh\nNever, never, never bring me back to your mind\nLet it slide, slide, roll off your mind\nLike I was a movie you'd seen\n\nAnd it was all just a crime, Virginia\nLet your fire be mercy to me\nI was a boy in your arms, safe in your arms\nAnd 

In [101]:
e = soup.find("div", class_="holder lyric-box")

In [108]:
text = e.getText().strip()
end = text.find("Edit Lyrics")

lyrics = text[:end]

In [133]:
songs["Lyrics"] = lyrics

In [134]:
songs["Artist"] = ARTIST

In [135]:
import os
songs.to_csv(os.getcwd()+"//songs.csv", index=False)

In [136]:
test = pd.read_csv(os.getcwd()+"//songs.csv")

In [137]:
print(test["Lyrics"][0])

Let it ride, let it ride, let it roll off your mind
Don't say a single world, get back inside
Let me drive, let me drive, disappear in the night
Like I was a ghost in your dreams

And it was all completely wrong, Virginia
You were much too much for me
I was a boy on the lawn, with his eyes to the sky
You were Garbo, smoke, and deceit

And it was Saturday, September, and fire
It was a backdrop set in your smile
It was all I could want in the sway of the song
I was haunted for so many years

Let it ride, let it ride, let it roll off your mind
Don't say a single world, get back inside
Let me drive, let me drive, disappear in the night
Like I was a ghost in your dreams

Say goodbye, say goodbye, give a kiss and a sigh
Never, never, never bring me back to your mind
Let it slide, slide, roll off your mind
Like I was a movie you'd seen

And it was all just a crime, Virginia
Let your fire be mercy to me
I was a boy in your arms, safe in your arms
And you swore like a promise to me

That I woul

In [116]:
songs.head()

,Album,Link,Title,Year,Lyrics
0,Señor and the Queen - EP,https:https://songmeanings.com/songs/view/3530...,Senor and the Queen,2008,"Let it ride, let it ride, let it roll off your..."
1,Señor and the Queen - EP,https:https://songmeanings.com/songs/view/3530...,"Wherefore Art Thou, Elvis?",2008,"Let it ride, let it ride, let it roll off your..."
2,Señor and the Queen - EP,https:https://songmeanings.com/songs/view/3530...,Say I Won't (Recognize),2008,"Let it ride, let it ride, let it roll off your..."
3,Señor and the Queen - EP,https:https://songmeanings.com/songs/view/3530...,Blue Jeans and White T-Shirts,2008,"Let it ride, let it ride, let it roll off your..."
4,Sink or Swim,https:https://songmeanings.com/songs/view/3530...,Boomboxes and Dictionaries,2008,"Let it ride, let it ride, let it roll off your..."


In [129]:
s = "10.10.2019"

In [130]:
try:
    year = int(s)
except ValueError:
    year = 0

In [131]:
year

0

In [132]:
s.isnumeric()

False

In [138]:
songs.describe()

,Album,Link,Title,Year,Lyrics,Artist
count,81,81,81,81,81,81
unique,8,76,76,5,1,1
top,Get Hurt,https:https://songmeanings.com/songs/view/3530...,The '59 Sound,2014,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
freq,16,2,2,26,81,81


In [139]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 6 columns):
Album     81 non-null object
Link      81 non-null object
Title     81 non-null object
Year      81 non-null object
Lyrics    81 non-null object
Artist    81 non-null object
dtypes: object(6)
memory usage: 1.9+ KB


In [144]:
songs[songs.Album=="Handwritten"]

,Album,Link,Title,Year,Lyrics,Artist
38,Handwritten,https:https://songmeanings.com/songs/view/3530...,,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
39,Handwritten,https:https://songmeanings.com/songs/view/3530...,Handwritten,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
40,Handwritten,https:https://songmeanings.com/songs/view/3530...,Here Comes My Man,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
41,Handwritten,https:https://songmeanings.com/songs/view/3530...,Mulholland Drive,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
42,Handwritten,https:https://songmeanings.com/songs/view/3530...,Keepsake,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
43,Handwritten,https:https://songmeanings.com/songs/view/3530...,Too Much Blood,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
44,Handwritten,https:https://songmeanings.com/songs/view/3530...,Howl,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
45,Handwritten,https:https://songmeanings.com/songs/view/3530...,Biloxi Parish,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
46,Handwritten,https:https://songmeanings.com/songs/view/3530...,Desire,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem
47,Handwritten,https:https://songmeanings.com/songs/view/3530...,Mae,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem


In [145]:
songs[songs.Title==""]

,Album,Link,Title,Year,Lyrics,Artist
38,Handwritten,https:https://songmeanings.com/songs/view/3530...,,2012,"Let it ride, let it ride, let it roll off your...",The Gaslight Anthem


In [146]:
link = "https://songmeanings.com/albums/view/tracks/275356/"

In [147]:
html = get_html(link)
soup = bs4.BeautifulSoup(html, "lxml")
songs = soup.find("table").find_all("a")
# for song in songs:
#     link = song.get_attribute_list("href")[0]
#     title = song.get_attribute_list("title")[0]
#     series = pd.Series({"Title": title, 
#                         "Link": link,
#                         "Album": row["Title"],
#                         "Year": row["Year"]})
#     df = df.append(series, ignore_index=True

In [150]:
songs[0].getText().strip()

'"45"'